In [1]:
import re
import os
import pandas as pd


greeks_list = ["\alpha","\beta","\gamma","\Gamma","\delta","\Delta","\epsilon","\varepsilon",
             "\zeta","\eta","\theta","\vartheta","\Theta","\iota","\kappa","\lambda","\Lambda",
             "\mu","\nu","\pi","\Pi","\rho","\varrho","\sigma","\Sigma","\tau","\phi","\varphi","\Phi",
             "\chi","\psi","\Psi","\omega","\Omega"]

# input a sample  tex
def generateDF(file, cat = "Unknown", name = 'Unknown', path = './CSVs/'):
    # clean up
    with open(file, 'r') as myfile:
        al = " ".join(myfile.readlines())

        brackets = re.compile('\[((.|\n)*?)\]')
        newal = brackets.sub('', al)

        comments = re.compile('\%.*')
        newal = comments.sub('\n', newal)
        
        whitespace = re.compile('(?!\w|\})\s+(?=\{)')
        newal = whitespace.sub('', newal)
        
        redundantNewlines = re.compile('\n\n')
        newal = redundantNewlines.sub('\n', newal)

    # write in clean up
    with open('cleanup.tex', 'w') as myfilecl:
        myfilecl.write(newal)
        
    with open('cleanup.tex', 'r') as myfile:
        i = 0
        matches = pd.DataFrame(columns = ['Line num','Command'])

        for idx, line in enumerate(myfile):
            i += 1
            match = re.findall(r'\\\w+', line)

            if '\\end' in match:
                match = []

            if '\\begin' in match:
                match = [re.findall('(?!\\begin{)\w+', line)[1]]
                
            for g in greeks_list:
                match[:] = [x for x in match if x != g]

            for idx, j in enumerate(match):
                new_row = pd.DataFrame({'Line num': i, 'Command': [j]})
                matches = matches.append(new_row, ignore_index=True)
                
    matches.to_csv(path + cat + '-' + name + '.csv')

In [2]:
# test
generateDF('sample.tex', name = 'sample', path = './')

In [3]:
unzipped_path = './Download/unzipped/'

categories = ["math.AC-CommutativeAlgebra", "math.AG-AlgebraicGeometry", "math.AP-AnalysisofPDEs", 
    "math.AT-AlgebraicTopology", "math.CA-ClassicalAnalysisandODEs", "math.CO-Combinatorics", "math.CT-CategoryTheory", 
    "math.CV-ComplexVariables", "math.DG-DifferentialGeometry", "math.DS-DynamicalSystems", "math.FA-FunctionalAnalysis", 
    "math.GM-GeneralMathematics", "math.GN-GeneralTopology", "math.GR-GroupTheory", "math.GT-GeometricTopology", 
    "math.HO-HistoryandOverview", "math.IT-InformationTheory", "math.KT-K-TheoryandHomology", "math.LO-Logic", 
    "math.MG-MetricGeometry", "math.MP-MathematicalPhysics", "math.NA-NumericalAnalysis", "math.NT-NumberTheory", 
    "math.OA-OperatorAlgebras", "math.OC-OptimizationandControl", "math.PR-Probability", "math.QA-QuantumAlgebra", 
    "math.RA-RingsandAlgebras", "math.RT-RepresentationTheory", "math.SG-SymplecticGeometry", "math.SP-SpectralTheory", "math.ST-StatisticsTheory"]

for cat in categories:
    cat_path = unzipped_path + cat
    file_paths = [x[0] for x in os.walk(cat_path)][1:]

    for file_path in file_paths:
        for file in os.listdir(file_path):
            if file.endswith(".tex"):
                try:
                    generateDF(file_path + '/' + file, cat[5:7], file[:-4])
                except:
                    pass